In [1]:
import numpy as np
from numpy.linalg import inv, det
import matplotlib.pyplot as plt
from copy import copy
import random
%matplotlib inline 

## Парето

In [2]:
def find_pareto_optimal(game_matrix):
    optimal_strats = []
    
    def is_optimal(row, col):
        input_item = game_matrix[row, col]
        
        for i in range(game_matrix.shape[0]):
            for j in range(game_matrix.shape[1]):
                cur_item = game_matrix[i, j]

                if (input_item <= cur_item).all() and(input_item < cur_item).any():
                    return False

        return True
    
    for i in range(game_matrix.shape[0]):
        for j in range(game_matrix.shape[1]):
            if is_optimal(i, j):
                optimal_strats.append((i, j))
                
    return optimal_strats            

## Нэш

In [197]:
# массив индексов максимального элемента
def max_item_indexes(vector):
    indexes = np.linspace(0, vector.shape[0] - 1, vector.shape[0], dtype = int)   
    
    return indexes[np.isin(vector, np.max(vector))]

def find_nash_optimal(game_matrix):
    # индексы наилучших стратегий первого игрока при i-й стратегии второго
    def first_max_ids(i):
        indexes = max_item_indexes(game_matrix[:, i, 0])
            
        return indexes[game_matrix[indexes, i, 1] >= np.max(game_matrix[indexes, i, 1])]
    
    # индексы наилучших стратегий второго игрока при i-й стратегии первого
    def second_max_ids(i):
        indexes = max_item_indexes(game_matrix[i, :, 1])
        
        return indexes[game_matrix[indexes, i, 0] >= np.max(game_matrix[indexes, i, 0])]
    
    optimal_strats = []
    
    for i in range(game_matrix.shape[0]):
        second_ids = second_max_ids(i)
        for j in second_ids:
            first_ids = first_max_ids(j)
            
            if i in first_ids:
                optimal_strats.append((i, j))
                    
    return optimal_strats

## Цветная печать

In [198]:
NASH_PREFIX = '\x1b[31m'
PARETO_PREFIX = '\x1b[32m'
BOTH_PREFIX = '\x1b[34m'
POSTFIX = '\x1b[0m'

def color_nash(string):
    return NASH_PREFIX + string + POSTFIX

def color_pareto(string):
    return PARETO_PREFIX + string + POSTFIX

def color_both(string):
    return BOTH_PREFIX + string + POSTFIX

def print_game_solution(game_matrix):
    print(color_nash('Nash optimal'))
    print(color_pareto('Pareto optimal'))
    print(color_both('Both optimal'))
    print()
    
    pareto = find_pareto_optimal(game_matrix)
    nash = find_nash_optimal(game_matrix)
    
    maxlen = 0
    
    for i in range(game_matrix.shape[0]):
        for j in range(game_matrix.shape[1]):
            maxlen = max(maxlen, len(str(game_matrix[i, j])))
    
    for i in range(game_matrix.shape[0]):
        row = []
        for j in range(game_matrix.shape[1]):
            item = (i, j)
            item_str = str(game_matrix[item])
            
            item_str = ' ' * (maxlen - len(item_str)) + item_str
            
            if item in pareto:
                if item in nash:
                    item_str = color_both(item_str)
                else:
                    item_str = color_pareto(item_str)
            elif item in nash:
                item_str = color_nash(item_str)

            row.append(item_str)
            
        print('  '.join(row))

## Условия

In [199]:
epsilon = round(random.randrange(10, 90)/100, 2)

crossroad = np.array(
   [[[1, 1], [1 - epsilon, 2]],
    [[2, 1 - epsilon], [0, 0]]]
)

family_conflict = np.array(
   [[[4, 1], [0, 0]],
    [[0, 0], [1, 4]]]
)

prisoners = np.array(
   [[[ -5, -5], [ 0, -10]],
    [[-10,  0], [-1,  -1]]]
)

var_15 = np.array(
   [[[0, 10], [9, 1]],
    [[7, 8], [6, 11]]]
)

In [200]:
print_game_solution(crossroad)

Nash optimal
Pareto optimal
Both optimal

    [1. 1.]  [0.58 2.  ]
[2.   0.58]      [0. 0.]


In [201]:
print_game_solution(family_conflict)

Nash optimal
Pareto optimal
Both optimal

[4 1]  [0 0]
[0 0]  [1 4]


In [202]:
print_game_solution(prisoners)

Nash optimal
Pareto optimal
Both optimal

  [-5 -5]  [  0 -10]
[-10   0]    [-1 -1]


In [203]:
print_game_solution(var_15)

Nash optimal
Pareto optimal
Both optimal

[ 0 10]    [9 1]
  [7 8]  [ 6 11]


In [204]:
# game_10 = np.random.randint(0, 10, (10, 10, 2))

In [205]:
print_game_solution(game_10)

Nash optimal
Pareto optimal
Both optimal

[5 6]  [1 4]  [5 9]  [6 4]  [8 9]  [8 7]  [1 1]  [3 5]  [1 0]  [9 7]
[9 5]  [9 0]  [7 5]  [0 0]  [7 8]  [2 2]  [3 1]  [5 4]  [4 9]  [6 8]
[8 7]  [7 2]  [2 7]  [7 3]  [1 5]  [7 3]  [3 4]  [5 7]  [6 7]  [0 7]
[3 7]  [5 4]  [4 6]  [1 5]  [4 7]  [1 0]  [6 0]  [5 5]  [9 4]  [6 1]
[7 9]  [5 7]  [8 3]  [2 2]  [6 5]  [5 0]  [8 7]  [6 0]  [5 8]  [0 6]
[8 5]  [0 2]  [0 6]  [7 6]  [8 4]  [4 9]  [2 8]  [2 3]  [9 9]  [2 1]
[9 8]  [3 1]  [3 2]  [8 8]  [3 0]  [7 0]  [2 7]  [8 3]  [8 1]  [4 2]
[6 4]  [1 4]  [7 4]  [8 7]  [5 8]  [9 2]  [9 5]  [3 4]  [4 7]  [3 2]
[4 2]  [3 0]  [5 6]  [3 7]  [9 6]  [6 1]  [7 3]  [9 5]  [2 5]  [0 5]
[8 1]  [2 7]  [8 2]  [3 4]  [1 5]  [3 7]  [9 9]  [3 0]  [4 3]  [4 1]


## Поиск смешанных стратегий

In [11]:
u = np.ones(2)

A = var_15[:,:,0]
B = var_15[:,:,1]

v1 = 1/(u.dot(inv(A)).dot(u))
v2 = 1/(u.dot(inv(B)).dot(u))

x = v2 * u.dot(inv(B))
y = v1 * u.dot(inv(A))

In [13]:
print('A:')
print(A)
print('B:')
print(B)
print()
print('v1:', round(v1, 3))
print('v2:', round(v2, 3))
print()
print('x:', x)
print('y:', y)

A:
[[0 9]
 [7 6]]
B:
[[10  1]
 [ 8 11]]

v1: 6.3
v2: 8.5

x: [0.25 0.75]
y: [0.1 0.9]


In [206]:
np.array([1, 2]) > np.array([2, 1])

array([False,  True])

In [208]:
np.array([1, 2]) > 1

array([False,  True])

In [207]:
np.array([1, 2])[[True, False]]

array([1])